LangChainMemory
```
LangChainMemory : 이전 대화 내용을 기억해서 문맥을 유지하는 역할 LangChain 0.3X부터는 LCEL 기반으로 체인을 구성,
RunnableWithMessageHistoru, ChatMessageHistory 등의 컴퍼넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이 문제를 해결결
```

In [1]:
%pip install langchain langchain-openai python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message('언녕하세요 제 이름은 이주영영입니다.')
history.add_ai_message('안녕하세요 이주영님, 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 언녕하세요 제 이름은 이주영영입니다.
ai : 안녕하세요 이주영님, 무엇을 도와드릴까요?


In [4]:
# Radis 기반 채팅 기록 저장소

from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv('REDIS_URL', 'redis://localhost:6379')
session_id = 'user_123'

history = RedisChatMessageHistory(session_id = session_id, redis_url = REDIS_URL)
history.add_user_message('안녕하세요 제 이름은 홍길동 입니다.')
history.add_ai_message('안녕하세요 홍길동님, 무엇을 도와드릴까요?')

# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')


ModuleNotFoundError: No module named 'langchain_redis'

In [7]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗복
# 핵심: session_id를 키로 하는 메모리 저장소 만들고 사용자의 대화는 키별로 저장한다
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",'당신은 뛰어난 한국어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요'),
    #history 키로 전달된 메세지 목록은 chain 실행시 해당 위치에 넣겠다는 의미
    MessagesPlaceholder(variable_name='history'), 
    ('human','{input}')
])
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0) 

In [8]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [5]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_sessino_history 를 구현

In [14]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
#세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id : str) -> InMemoryChatMessageHistory:
    '''"세션 ID"에 해당하는 대화 기록 객체를 반환합니다. (없으면 새로 생성)'''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼생성
chatbot = RunnableWithMessageHistory(
    runnable = chain,
    get_session_history = get_session_history,
    input_messages_key= 'input',
    history_messages_key= 'history'
)

In [15]:
# 두개의 세션을 번갈아가면서 대화 RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다
sessions = ['user_a','user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?', #user_a 첫번째 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', #user_b 첫 번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', #user_a 두 번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' #user_b 두 번째 질문
]
for i,question in enumerate(questions):
    session_id = sessions[i%2] # 세션 ID를 번갈아가며 사용 
    result = chatbot.invoke({'input':question}, config = {'configurable': {'session_id':session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 어떤 궁금한 점이나 도움이 필요하신 부분이 있으신가요?

[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 다양한 정보와 도움을 제공하는 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 멋지네요! 저는 여러분의 질문에 답변하고, 정보 제공, 문제 해결, 그리고 다양한 주제에 대한 상담을 하는 역할을 하고 있습니다. 프로그래밍에 관련된 질문이나 도움이 필요하시면 언제든지 말씀해 주세요! 어떤 언어를 배우고 계신가요?

[user_b] 질문: 저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?
[user_b] 챗봇: 역사에 관심이 많으시다니 정말 멋지네요! 저는 특정한 관심 분야가 없지만, 다양한 주제에 대한 정보를 제공할 수 있습니다. 역사, 과학, 기술, 문화, 예술 등 여러 분야에 대해 이야기할 수 있으니, 궁금한 점이나 더 알고 싶은 주제가 있다면 말씀해 주세요!



In [16]:
result = chatbot.invoke({'input':"저는 철수예요, 반갑습니다"}, config={'configurable' : {'session_id' : 'user_c'}})
print(f'[user_c] 질문: 저는 철수예요, 반갑습니다.')
print(f'[user_c] 챗봇: {result}\n')

[user_c] 질문: 저는 철수예요, 반갑습니다.
[user_c] 챗봇: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 것이 있다면 말씀해 주세요.



In [17]:
result = chatbot.invoke({'input' : '저는 누구라고요?'}, config = {'configurable': {'session_id' : 'user_c'}})
result
                        

'철수님이라고 하셨습니다! 맞나요? 더 궁금한 점이나 이야기하고 싶은 것이 있다면 언제든지 말씀해 주세요.'

요약 메모리 구현(대화내용 자동 요약)
```
긴 대화내용을 모두 프롬프트에 기록하는 것은 비 효율적 -> 프롬프트에 길이 제한에 걸릴 가능성이 있음
Conversation Summary Memory
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용
```
어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 남김
- 요약결과를 메모리에 시스템 메시지 등으로 저장 -> 메모리 절약
- 새로운 사용자 입력 시 요약된 맥락 + 최근 몇 메시지만 참고해서 llm 전달
```

In [19]:
# 요약용 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ('system','당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요'),
    ('human','{conversation}')   # 잔체 대화 내용을 하나의 문자열로 전달
])
# LCEL  
summary_chain = summary_prompt | llm | StrOutputParser()

In [20]:
# user_d 세션에 대화내용을 기록 간 대화 생성
long_queries = [
    '안녕, 오늘 우리 뭐하려고 했지?',
    '아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?',
    '그 회의에 누가 참석하는지 기억나니?',
    '단위 프로젝트 진행 상황도 공유해야 할까?',
    '최근에 이야기 했던 새로운 기능에 대한 업데이트는 있어?'
]
session_id = 'user_d'
for q in long_queries:
    answer = chatbot.invoke({'input':q}, config={'configurable' : {'session_id' : session_id}})

print(f'요약전 user_d의 메모리 메세지 개수: {len(store[session_id].messages)}')
print(store[session_id]) #요약하기전 대화 내용


요약전 user_d의 메모리 메세지 개수: 10
Human: 안녕, 오늘 우리 뭐하려고 했지?
AI: 안녕하세요! 오늘 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 하고 싶은 주제가 있다면 말씀해 주세요. 함께 이야기해보아요!
Human: 아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?
AI: 회의 시간이 정확히 몇 시인지 기억이 나지 않으신가요? 회의 일정은 보통 이메일이나 캘린더에 기록되어 있을 텐데, 확인해보시면 좋을 것 같아요. 만약 회의 준비에 도움이 필요하시다면 어떤 자료를 준비해야 하는지 말씀해 주시면 도와드릴 수 있습니다!
Human: 그 회의에 누가 참석하는지 기억나니?
AI: 회의에 참석하는 사람들을 기억하는 것은 중요하지만, 제가 그 정보를 알 수는 없어요. 보통 회의 초대장이나 이메일에 참석자 목록이 포함되어 있을 텐데, 그걸 확인해보시면 좋을 것 같아요. 참석자 목록을 알고 계신다면, 그들과의 논의 주제나 역할에 대해 이야기해드릴 수 있습니다! 도움이 필요하시면 말씀해 주세요.
Human: 단위 프로젝트 진행 상황도 공유해야 할까?
AI: 네, 단위 프로젝트 진행 상황을 공유하는 것은 좋은 아이디어입니다. 회의에서 프로젝트의 현재 상태, 진행 중인 작업, 완료된 작업, 그리고 앞으로의 계획 등을 공유하면 참석자들이 프로젝트에 대한 이해를 높이고, 필요한 피드백이나 지원을 받을 수 있습니다. 

또한, 진행 상황을 공유함으로써 팀원 간의 협업을 촉진하고, 문제를 조기에 발견하여 해결할 수 있는 기회를 제공할 수 있습니다. 어떤 내용을 포함할지 고민 중이라면, 구체적인 진행 상황이나 이슈, 다음 단계 등을 정리해보세요. 도움이 필요하시면 언제든지 말씀해 주세요!
Human: 최근에 이야기 했던 새로운 기능에 대한 업데이트는 있어?
AI: 새로운 기능에 대한 업데이트는 프로젝트의 진행 상황에 따라 다를 수 있습니다. 만약 그 기능에 대한 구체적인 내용이나 개발 상황을 알고 계신다면, 그 정보를 바탕으로 업데이트를 준비하시면 좋을 것 같

In [26]:
# 전체 대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원본 유지
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
# 요약 대상 대화 내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

# 1번 주고받은 대화 이후에 또 했다는 것 -> 마지막을 빼야함
if len(message) > 2 :
    original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message[ : -2]])
else : 
    original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message])

# llm으로 요약 생성
summary_text = summary_chain.invoke({'conversation' : original_dialog})
print("== 요약 내용 ==")
print(summary_text)
# 기존 메모리를 요약으로 교체 : 이전 내용 요약본 + 최근 QA 유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content = f'요약: {summary_text}'))

# 최근 대화의 마지막 QA쌍 복원
if len(message) >= 2 :
    last_user_msg = message[-2]
    last_ai_msg = message[-1]
    # 휴먼 메시지인지 검사
    if isinstance(last_user_msg, HumanMessage) :
        new_history.add_user_message(last_user_msg.content)
    else :
        new_history.messages.append(last_user_msg)
    if isinstance(last_ai_msg, AIMessage) :
        new_history.add_ai_message(last_ai_msg.content)
    else :
        new_history.messages.append(last_ai_msg)
# 메모리 교체
store[session_id] = new_history
    


== 요약 내용 ==
대화 요약: 
사용자는 내일 회의 자료를 준비해야 한다고 언급하며 회의 시간과 참석자를 확인하려고 한다. AI는 회의 일정과 참석자 목록을 이메일이나 캘린더에서 확인할 것을 제안하고, 단위 프로젝트 진행 상황을 회의에서 공유하는 것이 좋다고 조언한다. AI는 진행 상황 공유가 팀 협업과 문제 해결에 도움이 될 것이라고 강조하며, 필요한 경우 추가 도움을 제공할 준비가 되어 있다고 말한다.


최신 체인 구성 방법 v0.3
```
LLMChain, SequentialChain 등과 같이 클래스의존도를 줄임
Runnable 공통 인터페이스를 통해 일관성을 유지
핵심 : Runnable + Composition --> 프롬프트 | 모델 | 파서
```

In [2]:
%pip install langchain openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv
load_dotenv()  # .env파일의 내용을 읽어서 환경변수에 등록
import os
openai_key = os.getenv('OPENAI_API_KEY')
print(openai_key[:5]+'****')

sk-pr****


단일 체인 : prompt -> llm -> 출력파서 (상품설명)

In [7]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 템플릿을 정의
product_prompt = PromptTemplate.from_template(
    '제품 이름 : {product}\n'
    '이 제품의 특징과 장점을 잘 표현할 수 있는 한개의 문장으로 변동해 주세요'
)
print(product_prompt.format(product = '선풍기'))
# 2. 출력파서 정의
output_parser = StrOutputParser()

# 3. llm 정의
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature=0)

# 4. LCEL 체인 구성 프롬프트 -> 모델 -> 출력파서
product_chain = product_prompt | llm | output_parser  # runnable 객체

result = product_chain.invoke({'product' : '선풍기'})

print(result)

제품 이름 : 선풍기
이 제품의 특징과 장점을 잘 표현할 수 있는 한개의 문장으로 변동해 주세요
"강력한 바람과 조용한 작동으로 여름철 더위를 시원하게 날려주는 선풍기입니다."


In [9]:
 # # OpenAI는 Text Completion, 즉 문장을 이어서 완성하는 모델...
# # text-davinci 모델을 위한 라이브러리 이전방식이라, 역할기반 템플릿이 없고 단순한 prompt

# from langchain_openai import OpenAI
# llm = OpenAI(model='text-davinci-003', temperature=0.7)
# print(llm.invoke('다음 문장을 완성하세요: 옛날 옛적에 호랑이가 살았습니다.'))

# text-davinci는 작년 7월에 사라짐 

In [8]:
# OpenAI는 Text complation 문장을 이어서 완성하는 모델...
# text-davinci 모델을 위한 라이브러리 이전방식이라서, 역할기반 템플릿이 없고 단순한 prompt

from langchain_openai import OpenAI
llm = OpenAI(model='gpt-4o-mini',temperature=0.7)
print(llm.invoke('다음 문장을 완성히세요 : 옛날옛적에 호랑이가 살았습니다'))

. 그 호랑이는 사람을 잡아먹는 무서운 호랑이였습니다. 그래서 사람들은 호랑이를 피하기 위해서 ��에 들어가지 않았습니다. 그런데 어느 날, 호랑이는 배가 너무 고파져서 ��속으로 나가 사람을 잡아먹기로 결심했습니다. 호랑이는 ��속으로 들어가면서, "이제 나는 나의 맛있는 저녁을 찾을 거야!"라고 생각했습니다.

호랑이는 ��속 깊은 곳으로 들어가면서 여러 가지 동물들을 만났습니다. 사슴, 토끼, 그리고 다람���들이었습니다. 하지만 호랑이는 그들을 잡아먹지 않았습니다. 왜냐하면 그들은 너무 귀��고 무서워 보이지 않았기 때문입니다. 호랑이는 그 대신에 더 큰 먹이를 찾아 계속 ��속을 돌아다��습니다. 

그러다 호랑이는 한 마을을 발견했습니다. 호랑이는 그 마을로 들어가서 사람들을 찾아봤습니다. 그런데 마을 사람들은 호랑이를 보고 너무 놀라서 도망가버렸습니다. 호랑이는 "이렇게 쉽게 사람들을 잡을 수 있다니!"라며


다중체인 : 체인 합성 및 Runnable 병합 (이메일 생성)
```
둘 이상의 llm 호출을 연결해서 복잡한 작업을 수행
'주어진 상황에 대한 이메일 작성' --> 제목
제목을 활용해서 이메일 본문을 작성 --> 본문
chain composition
```

In [10]:
# 프롬프트 1 : 사용자로부터 받은 이메일 요청 내용을 입력받아서 '이메일 제목'을 한문장으로 생성하는 명령
# llm 호출 -> 이메일 제목 출력('프로젝트 진행상황 회의 일정 안내')
# 중간 출력 변환 : 생성된 제목문자열을 {subject} 키를 갖는 dictionary 변환
# 프롬프트 2 : {subject} 변수를 받아서 해당 제목을 가진 이메일 본문내용을 요청
# llm 호출

In [14]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요'
    '{content}'
)
# 2. 이메일 본문 생성용 프롬프트
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 본문을 작성해 주세요\n'
    '제목 : {subject}\n'
    '본문 :'
)
# 3. 두 프롬프트를 결합한 체인 구성
email_chain = (
    subject_prompt
    | llm
    | {'subject' : RunnablePassthrough()}  # 출력된 제목을 subject 키로 매핑
    | body_prompt
    | llm
    | StrOutputParser()
)
# 4. 실행
result = email_chain.invoke({
    'content' : '다음주 단위 프로젝트를 진행하기 위해 팀 회의를 요청하는 이메일'
})
print(result)

  다음주 단위 프로젝트 팀 회의 요청

안녕하세요 팀원 여러분,

다음 주 단위 프로젝트를 위한 팀 회의를 요청드립니다. 프로젝트의 진행 상황을 점검하고, 각자의 역할을 정리하여 보다 효율적으로 작업을 이어나가기 위해 꼭 필요한 시간이 될 것입니다.

회의에서 다�� 주요 내용은 다음과 같습니다:
1. 프로젝트 목표 및 일정 확인
2. 역할 분담 및 책임 확인
3. 진행 상황 점검 및 피드백 공유

모든 팀원 여러분의 소중한 의견이 필요합니다. 아래의 가능한 일정을 알려주시면 감사하겠습니다.

- 날짜 및 시간:
- 기타 제안 사항:

여러분의 협조에 감사드리며, 회의에서 ���기를 기대합니다.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[당신의 연락처]  


In [20]:
from langchain_core.runnables import RunnablePassthrough
# 1. 이메일 생성용 프롬프트 정의
subject_prompt = PromptTemplate.from_template(
    '다음 요청 내용을 바탕으로 이메일 제목을 만들어주세요\n'
    '{content}'
)
# 2. 이메일 본문 생성용 프롬프트
body_prompt = PromptTemplate.from_template(
    '위에서 생성된 제목을 활용해서 팀에게 보내는 정중한 이메일 본문을 작성해 주세요\n'
    '제목:{subject}\n'
    '본문:'
)
# 3. 두 프롬프트를 결합한 체인을 구성
email_chain = (
    subject_prompt
    | llm
    | {'subject' : RunnablePassthrough()} # 출력된 제목을 subject 키로 매핑
    | body_prompt
    | llm
    | StrOutputParser()
)
# 4. 실행
result = email_chain.invoke({
    'content' : '다음주 단위 프로젝트를 진행하기 위해 팀 회의를 요청하는 이메일'
})
print(result)

 다음주 단위 프로젝트 팀 회의 요청

제목: 다음주 단위 프로젝트 팀 회의 요청

안녕하세요, 팀 여러분.

다음주에 진행할 단위 프로젝트에 대해 논의하기 위해 팀 회의를 요청드립니다. 모든 팀원들이 참석할 수 있는 최적의 일정을 조율하고자 합니다.

가능하신 날짜와 시간을 댓글로 남겨주시면, 가장 많은 분들이 참석할 수 있는 시간으로 회의를 잡도록 하겠습니다.

감사합니다.

[당신의 이름]  
[당신의 직책]  
[당신의 연락처]

---

이메일 내용이 적절한지 확인하시고, 필요에 따라 수정해 주세요! 추가로 요청하실 사항이 있으시면 언제든지 말씀해 주세요. 감사합니다!


조건분기

In [23]:
from langchain_core.runnables import RunnableBranch, RunnableLambda
# 1. 요약체인(prompt->llm)
summary_prompt = PromptTemplate.from_template(
    '다음 문장을 한 문단으로 간결하게 요약해주세요\n'
    '{text}'
)
summary_chain = summary_prompt | llm | StrOutputParser()
# 2. email chain은 재활용
# 3. 분기조건 함수 정의 -> Runnable로 래핑
def is_summary_request(user_input:str) -> bool:
    return user_input.strip().startswith('요약:')
condition = RunnableLambda(is_summary_request)
# 분기 체인
branch_chain = RunnableBranch(
    (condition, summary_chain), email_chain
)
# 다양한 조건
input1 = '요약: 오늘 회의에서는 다양한 주제에 대한 토론이 있었는데, 특히 일정 변경과 예산 관련 내용이 다수였습니다.'
input2 = '이메일: 다음주 월요일 프로젝트 회의 일정을 팀에게 공지해줘'

result1 = branch_chain.invoke(input1)
result2 = branch_chain.invoke(input2)
print(f'result1: {result1}')
print(f'result2: {result2}')

result1:  참석자들은 변경 사항에 대한 의견을 나누었고, 예산 조정의 필요성에 대해 공감했습니다.

오늘 회의에서는 일정 변경과 예산 조정에 대한 의견이 활발히 논의되었습니다. 참석자들은 이 변경 사항에 대해 의견을 교환하며 예산 조정의 필요성에 공감했습니다.
result2:  
안녕하세요 팀원 여러분,

다음주 월요일에 예정된 프로젝트 회의에 대한 일정을 안내드립니다. 이번 회의에서는 현재 진행 중인 프로젝트의 진행 상황을 점검하고, 향후 계획에 대해 논의할 예정입니다.

회의는 다음과 같은 일정으로 진행됩니다:
- 일시: 다음주 월요일, 오후 2시
- 장소: 3층 회의실 A

회의에 참석하실 수 있는 모든 분들의 참여를 부탁드립니다. 회의 준비를 위해 각자 맡은 부분의 진행 상황을 정리해오시면 감사하겠습니다.

궁금한 사항이 있으시면 언제든지 저에게 문의해 주세요.

감사합니다.

[당신의 이름]  
[당신의 직함]  
[회사 이름]  
[연락처]  
